## Nobel API - check Wikidata and Nobel API
version. 0.2 changed api call and added **Vrandečić EWS**
* This [Jupyter Notebook](https://github.com/salgo60/open-data-examples/blob/master/Nobel%20API.ipynb)  

  * WIkidata property [Property:P8024](https://www.wikidata.org/wiki/Property:P8024)
  * [Developer xone nobelprize.org](https://www.nobelprize.org/about/developer-zone-2/)
      * maybe better API is [api.nobelprize.org/2.0/laureates](https://api.nobelprize.org/2.0/laureates)

In [1]:
from datetime import datetime
start_time = datetime.now()
print("Last run: ", datetime.now())

Last run:  2020-10-09 00:02:31.271201


In [2]:
import urllib3, json
import pandas as pd 
http = urllib3.PoolManager() 

url ="https://api.nobelprize.org/2.0/laureates?offset=0&limit=1500"
dftot = pd.DataFrame()
r = http.request('GET', url)
data = json.loads(r.data)
dftot = dftot.append(pd.DataFrame(data["laureates"]),sort=False)


In [3]:
dftot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 952 entries, 0 to 951
Data columns (total 18 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   id               952 non-null    object
 1   knownName        928 non-null    object
 2   givenName        928 non-null    object
 3   familyName       926 non-null    object
 4   fullName         928 non-null    object
 5   gender           928 non-null    object
 6   birth            927 non-null    object
 7   links            952 non-null    object
 8   nobelPrizes      952 non-null    object
 9   death            628 non-null    object
 10  orgName          24 non-null     object
 11  nativeName       24 non-null     object
 12  acronym          10 non-null     object
 13  founded          23 non-null     object
 14  penName          11 non-null     object
 15  birthCountry     1 non-null      object
 16  birthCountryNow  1 non-null      object
 17  birthContinent   1 non-null      ob

In [4]:
dftot["id"] = dftot["id"].astype("int")

dftot.sort_values(by=['id'], ascending=[True], inplace=True)
dftot.tail(10)


,id,knownName,givenName,familyName,fullName,gender,birth,links,nobelPrizes,death,orgName,nativeName,acronym,founded,penName,birthCountry,birthCountryNow,birthContinent
602,984,"{'en': 'Michael Kremer', 'se': 'Michael Kremer'}","{'en': 'Michael', 'se': 'Michael'}","{'en': 'Kremer', 'se': 'Kremer'}","{'en': 'Michael Kremer', 'se': 'Michael Kremer'}",male,"{'date': '1964-11-12', 'place': {'city': {'en'...","{'rel': 'laureate', 'href': 'http://masterdata...","[{'awardYear': '2019', 'category': {'en': 'Eco...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
352,985,"{'en': 'Harvey J. Alter', 'se': 'Harvey J. Alt...","{'en': 'Harvey', 'se': 'Harvey'}","{'en': 'Alter', 'se': 'Alter'}","{'en': 'Harvey J. Alter', 'se': 'Harvey J. Alt...",male,"{'date': '1935-00-00', 'place': {'city': {'en'...","{'rel': 'laureate', 'href': 'http://masterdata...","[{'awardYear': '2020', 'category': {'en': 'Phy...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
601,986,"{'en': 'Michael Houghton', 'se': 'Michael Houg...","{'en': 'Michael', 'se': 'Michael'}","{'en': 'Houghton', 'se': 'Houghton'}","{'en': 'Michael Houghton', 'se': 'Michael Houg...",male,NaN,"{'rel': 'laureate', 'href': 'http://masterdata...","[{'awardYear': '2020', 'category': {'en': 'Phy...",NaN,NaN,NaN,NaN,NaN,NaN,"{'en': 'United Kingdom', 'no': 'Storbritannia'...","{'en': 'United Kingdom', 'no': 'Storbritannia'...",{'en': 'Europe'}
132,987,"{'en': 'Charles M. Rice', 'se': 'Charles M. Ri...","{'en': 'Charles', 'se': 'Charles'}","{'en': 'Rice', 'se': 'Rice'}","{'en': 'Charles M. Rice', 'se': 'Charles M. Ri...",male,"{'date': '1952-00-00', 'place': {'city': {'en'...","{'rel': 'laureate', 'href': 'http://masterdata...","[{'awardYear': '2020', 'category': {'en': 'Phy...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
770,988,"{'en': 'Roger Penrose', 'se': 'Roger Penrose'}","{'en': 'Roger', 'se': 'Roger'}","{'en': 'Penrose', 'se': 'Penrose'}","{'en': 'Roger Penrose', 'se': 'Roger Penrose'}",male,"{'date': '1931-00-00', 'place': {'city': {'en'...","{'rel': 'laureate', 'href': 'http://masterdata...","[{'awardYear': '2020', 'category': {'en': 'Phy...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
715,989,"{'en': 'Reinhard Genzel', 'se': 'Reinhard Genz...","{'en': 'Reinhard', 'se': 'Reinhard'}","{'en': 'Genzel', 'se': 'Genzel'}","{'en': 'Reinhard Genzel', 'se': 'Reinhard Genz...",male,"{'date': '1952-03-24', 'place': {'city': {'en'...","{'rel': 'laureate', 'href': 'http://masterdata...","[{'awardYear': '2020', 'category': {'en': 'Phy...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
54,990,"{'en': 'Andrea Ghez', 'se': 'Andrea Ghez'}","{'en': 'Andrea', 'se': 'Andrea'}","{'en': 'Ghez', 'se': 'Ghez'}","{'en': 'Andrea Ghez', 'se': 'Andrea Ghez'}",female,"{'date': '1965-00-00', 'place': {'city': {'en'...","{'rel': 'laureate', 'href': 'http://masterdata...","[{'awardYear': '2020', 'category': {'en': 'Phy...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
216,991,"{'en': 'Emmanuelle Charpentier', 'se': 'Emmanu...","{'en': 'Emmanuelle', 'se': 'Emmanuelle'}","{'en': 'Charpentier', 'se': 'Charpentier'}","{'en': 'Emmanuelle Charpentier', 'se': 'Emmanu...",female,"{'date': '1968-12-11', 'place': {'city': {'en'...","{'rel': 'laureate', 'href': 'http://masterdata...","[{'awardYear': '2020', 'category': {'en': 'Che...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
448,992,"{'en': 'Jennifer A. Doudna', 'se': 'Jennifer A...","{'en': 'Jennifer A.', 'se': 'Jennifer A.'}","{'en': 'Doudna', 'se': 'Doudna'}","{'en': 'Jennifer A. Doudna', 'se': 'Jennifer A...",female,"{'date': '1964-02-19', 'place': {'city': {'en'...","{'rel': 'laureate', 'href': 'http://masterdata...","[{'awardYear': '2020', 'category': {'en': 'Che...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
559,993,"{'en': 'Louise Glück', 'se': 'Louise Glück'}","{'en': 'Louise', 'se': 'Louise'}","{'en': 'Glück', 'se': 'Glück'}","{'en': 'Louise Glück', 'se': 'Louise Glück'}",female,"{'date': '1943-00-00', 'place': {'city': {'en'...","{'rel': 'laureate', 'href': 'http://masterdata...","[{'awardYear': '2020', 'category': {'en': 'Lit...",NaN,NaN,NaN,

In [5]:
#dftot.sample(10)

In [6]:
listNobel = []
for index,row in dftot.iterrows():
    new_item = dict()
    new_item['id'] = row['id']
    try:
        new_item['name_en'] = row['knownName']['en']
    except:
        pass
    try:
        new_item['name_se'] = row['knownName']['se']
    except:
        pass
    new_item['link'] = row['links']['href']
    try:
        new_item['awardYear'] = row['nobelPrizes'][0]['awardYear']
        new_item['category'] = row['nobelPrizes'][0]['category']['en']
    except:
        pass
    try:
        new_item['gender'] = row['gender']
    except:
        pass
    listNobel.append(new_item)
NobelTot = pd.DataFrame(listNobel,
                  columns=['id','name_en','name_se','link','gender','awardYear','category'])

NobelTot.info()    


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 952 entries, 0 to 951
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         952 non-null    int64 
 1   name_en    928 non-null    object
 2   name_se    928 non-null    object
 3   link       952 non-null    object
 4   gender     928 non-null    object
 5   awardYear  952 non-null    object
 6   category   952 non-null    object
dtypes: int64(1), object(6)
memory usage: 52.2+ KB


## EWS Vrandečić  
* [tweet](https://twitter.com/vrandezo/status/1313587353433251840?s=20)

In [7]:
NobelTot[NobelTot['name_en'].str.contains("Vrandečić", na=False)] 

,id,name_en,name_se,link,gender,awardYear,category


In [8]:
#check number of prizes received
NobelTot[NobelTot['name_en'].str.contains("Vrandečić", na=False)]["name_en"].sum()


0

## Check Wikidata matches

In [9]:
# pip install sparqlwrapper
# https://rdflib.github.io/sparqlwrapper/

import sys
from SPARQLWrapper import SPARQLWrapper, JSON

endpoint_url = "https://query.wikidata.org/sparql"

query = """SELECT ?item ?itemLabel ?NobelAPI WHERE {
  ?item wdt:P8024 ?NobelAPI.
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
} order by xsd:integer(?NobelAPI)"""


def get_sparql_dataframe(endpoint_url, query):
    """
    Helper function to convert SPARQL results into a Pandas data frame.
    """
    user_agent = "salgo60/%s.%s" % (sys.version_info[0], sys.version_info[1])
 
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    result = sparql.query()

    processed_results = json.load(result.response)
    cols = processed_results['head']['vars']
    out = []
    for row in processed_results['results']['bindings']:
        item = []
        for c in cols:
            item.append(row.get(c, {}).get('value'))
        out.append(item)

    return pd.DataFrame(out, columns=cols)

WDNobel = get_sparql_dataframe(endpoint_url, query)

In [10]:
WDNobel["NobelAPI"] = WDNobel["NobelAPI"].astype("int")
WDNobel.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 952 entries, 0 to 951
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   item       952 non-null    object
 1   itemLabel  952 non-null    object
 2   NobelAPI   952 non-null    int64 
dtypes: int64(1), object(2)
memory usage: 22.4+ KB


### Compare WD <-> Nobel 

In [11]:
dfmerge = pd.merge(WDNobel, NobelTot,how='outer', left_on="NobelAPI",right_on="id",indicator=True)
dfmerge['_merge'] = dfmerge['_merge'].str.replace('left_only','WD_only').str.replace('right_only','Nobel_only')
dfmerge.rename(columns={"_merge": "WD_Nobel"},inplace = True)    
dfmerge[-10:]  

,item,itemLabel,NobelAPI,id,name_en,name_se,link,gender,awardYear,category,WD_Nobel
942,http://www.wikidata.org/entity/Q1635006,Michael Kremer,984,984,Michael Kremer,Michael Kremer,http://masterdataapi.nobelprize.org/2/laureate...,male,2019,Economic Sciences,both
943,http://www.wikidata.org/entity/Q5677307,Harvey J. Alter,985,985,Harvey J. Alter,Harvey J. Alter,http://masterdataapi.nobelprize.org/2/laureate...,male,2020,Physiology or Medicine,both
944,http://www.wikidata.org/entity/Q1927895,Michael Houghton,986,986,Michael Houghton,Michael Houghton,http://masterdataapi.nobelprize.org/2/laureate...,male,2020,Physiology or Medicine,both
945,http://www.wikidata.org/entity/Q22669757,Charles M. Rice,987,987,Charles M. Rice,Charles M. Rice,http://masterdataapi.nobelprize.org/2/laureate...,male,2020,Physiology or Medicine,both
946,http://www.wikidata.org/entity/Q193803,Roger Penrose,988,988,Roger Penrose,Roger Penrose,http://masterdataapi.nobelprize.org/2/laureate...,male,2020,Physics,both
947,http://www.wikidata.org/entity/Q65807,Reinhard Genzel,989,989,Reinhard Genzel,Reinhard Genzel,http://masterdataapi.nobelprize.org/2/laureate...,male,2020,Physics,both
948,http://www.wikidata.org/entity/Q493956,Andrea M. Ghez,990,990,Andrea Ghez,Andrea Ghez,http://masterdataapi.nobelprize.org/2/laureate...,female,2020,Physics,both
949,http://www.wikidata.org/entity/Q17280087,Emmanuelle Charpentier,991,991,Emmanuelle Charpentier,Emmanuelle Charpentier,http://masterdataapi.nobelprize.org/2/laureate...,female,2020,Chemistry,both
950,http://www.wikidata.org/entity/Q56068,Jennifer Doudna,992,992,Jennifer A. Doudna,Jennifer A. Doudna,http://masterdataapi.nobelprize.org/2/laureate...,female,2020,Chemistry,both
951,http://www.wikidata.org/entity/Q2344210,Louise Glück,993,993,Louise Glück,Louise Glück,http://masterdataapi.nobelprize.org/2/laureate...,female,2020,Literature,both


In [12]:
dfmerge["WD_Nobel"].value_counts()  

both    952
Name: WD_Nobel, dtype: int64

In [13]:
Nobelonly = dfmerge[dfmerge["WD_Nobel"]=="Nobel_only"].copy() 

from IPython.display import HTML
Nobelonly["Nobel"] = "<a href='https://api.nobelprize.org/v1/laureate.json?id=" + Nobelonly["id"] + "'>link</a>"
Nobelonly["Nobelhtml"] = "<a href='https://www.nobelprize.org/laureate/" + Nobelonly["id"] + "'>html</a>"
pd.set_option("display.max.columns", None) 
HTML(Nobelonly.to_html(escape=False))

,item,itemLabel,NobelAPI,id,name_en,name_se,link,gender,awardYear,category,WD_Nobel,Nobel,Nobelhtml


In [14]:
WDonly = dfmerge[dfmerge["WD_Nobel"]=="WD_only"].copy() 
WDonly

,item,itemLabel,NobelAPI,id,name_en,name_se,link,gender,awardYear,category,WD_Nobel


In [15]:
print("End run: ", datetime.now())

End run:  2020-10-09 00:02:34.074029


In [16]:
print('Time elapsed (hh:mm:ss.ms) {}'.format(datetime.now() - start_time))

Time elapsed (hh:mm:ss.ms) 0:00:02.807594


In [17]:
# see task xxx we can create links JSOn and HTML using the ID
dfmerge["linkjson"] = "<a href='https://api.nobelprize.org/v1/laureate.json?id=" + dfmerge["id"].astype(str) + "'>json</a>"
dfmerge["linkhtml"] = "<a href='https://www.nobelprize.org/laureate/" + dfmerge["id"].astype(str) + "'>html</a>"
dfmerge["wd"] = "<a href='" + dfmerge["item"].astype(str) + "'>Wikidata</a>"

pd.set_option("display.max.columns", None) 
HTML(dfmerge[{'wd','name_en','awardYear','category','linkjson','linkhtml'}].tail(10).to_html(escape=False))

,linkjson,wd,linkhtml,awardYear,category,name_en
942,json,Wikidata,html,2019,Economic Sciences,Michael Kremer
943,json,Wikidata,html,2020,Physiology or Medicine,Harvey J. Alter
944,json,Wikidata,html,2020,Physiology or Medicine,Michael Houghton
945,json,Wikidata,html,2020,Physiology or Medicine,Charles M. Rice
946,json,Wikidata,html,2020,Physics,Roger Penrose
947,json,Wikidata,html,2020,Physics,Reinhard Genzel
948,json,Wikidata,html,2020,Physics,Andrea Ghez
949,json,Wikidata,html,2020,Chemistry,Emmanuelle Charpentier
950,json,Wikidata,html,2020,Chemistry,Jennifer A. Doudna
951,json,Wikidata,html,2020,Literature,Louise Glück


Status 20201008 :
* 945   Charles M. Rice - missing html
* 950	Jennifer A. Doudna - missing html
* 951	Louise Glück - missing html

Templates supporting Wikidata Property 8024 = Wikidata [Q91652187](https://www.wikidata.org/wiki/Q91652187)
* ar:Wikipedia [قالب:جائزة_نوبل](https://ar.wikipedia.org/wiki/%D9%82%D8%A7%D9%84%D8%A8:%D8%AC%D8%A7%D8%A6%D8%B2%D8%A9_%D9%86%D9%88%D8%A8%D9%84)
* ca:wikipedia [Plantilla:Nobelprize](https://ca.wikipedia.org/wiki/Plantilla:Nobelprize)
* en:Wikipedia [Template:Nobelprize](https://en.wikipedia.org/wiki/Template:Nobelprize)
* ka:wikipedia [თარგი:Nobelprize](https://ka.wikipedia.org/wiki/%E1%83%97%E1%83%90%E1%83%A0%E1%83%92%E1%83%98:Nobelprize)
* nn:Wikipedia [Mal:Nobelprize](https://nn.wikipedia.org/wiki/Mal:Nobelprize)
* pt:Wikipedia [Predefinição:Prémio Nobel](https://pt.wikipedia.org/wiki/Predefini%C3%A7%C3%A3o:Pr%C3%A9mio_Nobel)
* sv:Wikipedia [Mall:Nobelprize](https://sv.wikipedia.org/wiki/Mall:Nobelprize)
* sr:WIkipedia [Шаблон:Nobelprize](https://sr.wikipedia.org/wiki/%D0%A8%D0%B0%D0%B1%D0%BB%D0%BE%D0%BD:Nobelprize)

# Next announcement
see https://www.nobelprize.org/